In [ ]:
import pickle
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import time
import gc
import numpy as np
import lightgbm as lgb
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import torch
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_curve 
import time
import os
import itertools
import random
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.special import erfinv
from collections import OrderedDict
from math import sqrt
import numpy as np
from torch.optim import lr_scheduler
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import lightgbm as lgb


In [ ]:
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder as Encoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders import CountEncoder
from category_encoders.one_hot import OneHotEncoder

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [ ]:
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

In [ ]:

def lower_sample_data_by_sample(df,percent=1,rs=42):
    most_data = df[df['label'] == 0]  # 多数类别的样本
    minority_data = df[df['label'] == 1]  # 少数类别的样本   
    #随机采样most_data中的数据
    lower_data=most_data.sample(n=int(percent*len(minority_data)),replace=False,random_state=rs,axis=0)   
    return (pd.concat([lower_data,minority_data]))

In [ ]:
def get_mask_train(df,samp):
  if random.random()<samp:
    return -1
  else :
    return df

In [ ]:
#------------------------------数据预处理----------------------------------------#

In [ ]:
columns = [ 'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
       'communication_onlinerate', 'communication_avgonline_30d', 'indu_name',
       'pt_d']

In [ ]:
#                读取数据集
train_df = reduce_mem(pd.read_csv('train_data.csv',sep='|'))
#train_df = train_df.drop_duplicates(subset=columns)

test_df = pd.read_csv('test_data_B.csv',sep='|')
test_df['pt_d']-=1

In [ ]:
test_df_A = pd.read_csv('test_data_A.csv',sep='|')

In [ ]:
# 无用列
drop_cols = ['pt_d','label','communication_onlinerate','index','id','K']

# 选择类别特征
cat_cols = [ 'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase'
        , 'communication_avgonline_30d', 'indu_name',
      ]
MASK = 'MASK'
miss_col1 = ['task_id', 'adv_id','uid']
miss_col2 = ['adv_prim_id','dev_id' ]#, 'device_size','spread_app_id','indu_name']

In [ ]:
train_df.reset_index(drop=True,inplace=True)

In [ ]:
count_history_cols = rh_col =  [ 'uid', 'task_id', 'adv_id']

In [ ]:
test_df = pd.concat([test_df_A,test_df])

In [ ]:
train_df['id'] = 0
train_df['first_online'] = train_df['communication_onlinerate'].apply(lambda x:int(x.split('^')[0]))
train_df['last_online'] = train_df['communication_onlinerate'].apply(lambda x:int(x.split('^')[-1]))
test_df['first_online'] = test_df['communication_onlinerate'].apply(lambda x:int(x.split('^')[0]))
test_df['last_online'] = test_df['communication_onlinerate'].apply(lambda x:int(x.split('^')[-1]))

In [ ]:
def get_tfidf(train,test,key1,key2):
    
    train_tif = pd.DataFrame(train[[key1, key2]].groupby([key1])[key2].apply(list))
    train_tif.reset_index(inplace=True)
    train_key1= train_tif[key1].values
    train_key2    = train_tif[key2].values.tolist()
    train_key2_list = []
    for seq in train_key2:
        sentences = []
        for word in seq:
            sentences.append(str(word))
        train_key2_list.append(' '.join(sentences))
    

    tfidf_vec = TfidfVectorizer() 
    train_tfidf_matrix = tfidf_vec.fit_transform(train_key2_list).toarray()

    test_tif = pd.DataFrame(test[[key1, key2]].groupby([key1])[key2].apply(list))
    test_tif.reset_index(inplace=True)
    test_key1= test_tif[key1].values
    test_key2 = test_tif[key2].values.tolist()
    test_key2_list = []
    for seq in test_key2:
        sentences = []
        for word in seq:
            sentences.append(str(word))
        test_key2_list.append(' '.join(sentences))
    test_tfidf_matrix = tfidf_vec.transform(test_key2_list).toarray()
    assert train_tfidf_matrix.shape[1]==test_tfidf_matrix.shape[1]
    
    train_tfidf_agmax = np.argmax(train_tfidf_matrix,axis=1)
    train_tfidf_max = np.max(train_tfidf_matrix,axis=1)
    train_tfidf_mean = np.mean(train_tfidf_matrix,axis=1)
    train_tfidf_std = np.std(train_tfidf_matrix,axis=1)
    
    test_tfidf_agmax = np.argmax(test_tfidf_matrix,axis=1)
    test_tfidf_max = np.max(test_tfidf_matrix,axis=1)
    test_tfidf_mean = np.mean(test_tfidf_matrix,axis=1)
    test_tfidf_std = np.std(test_tfidf_matrix,axis=1)
    
    print('train_tfidf_agmax.shape:')
    print(train_tfidf_agmax.shape)
    
    print('train_tfidf_mean.shape:')
    print(train_tfidf_mean.shape)
    
    print('test_tfidf_agmax.shape:')
    print(test_tfidf_agmax.shape)
    
    print('test_tfidf_mean.shape:')
    print(test_tfidf_mean.shape)
    return train_tif,test_tif,train_tfidf_agmax,train_tfidf_max,train_tfidf_mean,train_tfidf_std,test_tfidf_agmax,test_tfidf_max,test_tfidf_mean,test_tfidf_std

In [ ]:
def get_temporal_feature(train,test,col):
    
    print('统计昨天点击数')
    
    label_sum = train.groupby([col,'pt_d'])['label'].agg('sum').reset_index()
    label_sum.columns = [col,'pt_d',col+'yesterday_label_sum']
    label_sum['pt_d']+=1
    test = test.merge(label_sum,on = [col,'pt_d'],how = 'left')
    train = train.merge(label_sum,on = [col,'pt_d'],how = 'left')
    
    print('统计今天曝光数')
    
    today_count_train = train.groupby([col,'pt_d'])['id'].agg('count').reset_index()
    today_count_train.columns = [col,'pt_d',col+'today_expos_count']
    train = train.merge(today_count_train,on = [col,'pt_d'],how = 'left')

    today_count_test = test.groupby([col,'pt_d'])['id'].agg('count').reset_index()
    today_count_test.columns = [col,'pt_d',col+'today_expos_count']
    test = test.merge(today_count_test,on = [col,'pt_d'],how = 'left')
    test[col+'today_expos_count']*=3
    
    print('统计昨天曝光数')
    
    expos = train.groupby([col,'pt_d'])['id'].agg('count').reset_index()
    expos.columns = [col,'pt_d',col+'yesterday_expos_count']
    expos['pt_d']+=1
    test = test.merge(expos,on = [col,'pt_d'],how = 'left')
    train = train.merge(expos,on = [col,'pt_d'],how = 'left')
    
    return train,test

In [ ]:
for col in count_history_cols:
    train_df,test_df = get_temporal_feature(train_df,test_df,col)

In [ ]:
# train_df = train_df[train_df.pt_d!=1].reset_index(drop=True)
test_df = test_df.iloc[1000000:]

In [ ]:

# train_tif_uid1,test_tif_uid1,train_tfidf_agmax,train_tfidf_max,train_tfidf_mean,train_tfidf_std,test_tfidf_agmax,test_tfidf_max,test_tfidf_mean,test_tfidf_std = get_tfidf(train_df , test_df, 'uid','task_id')
# train_tif_uid1 = train_tif_uid1.drop('task_id',axis=1)
# train_tif_uid1['uid'+'task_id'+'tf_argmax'] = train_tfidf_agmax
# train_tif_uid1['uid'+'task_id'+'max'] = train_tfidf_max
# train_tif_uid1['uid'+'task_id'+'mean'] = train_tfidf_mean
# train_tif_uid1['uid'+'task_id'+'std'] = train_tfidf_std

# train_tif_uid2,test_tif,train_tfidf_agmax,train_tfidf_max,train_tfidf_mean,train_tfidf_std,test_tfidf_agmax,test_tfidf_max,test_tfidf_mean,test_tfidf_std = get_tfidf(train_df , test_df, 'uid','adv_id')
# train_tif_uid2 = train_tif_uid2.drop('adv_id',axis=1)
# train_tif_uid2['uid'+'adv_id'+'tf_argmax'] = train_tfidf_agmax
# train_tif_uid2['uid'+'adv_id'+'max'] = train_tfidf_max
# train_tif_uid2['uid'+'adv_id'+'mean'] = train_tfidf_mean
# train_tif_uid2['uid'+'adv_id'+'std'] = train_tfidf_std

In [ ]:
feature = [i for i in train_df.columns if i not in drop_cols]

In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
#------------------------------模型训练----------------------------------------#

In [ ]:
feature_importance_df = pd.DataFrame()
predicts = np.zeros(len(train_df))
pred = np.zeros(len(test_df))
true = np.zeros(len(train_df))
params = {'learning_rate': 0.1,
         'boosting': 'gbdt',
         'objective':'binary',
         'boosting': 'gbdt',
         'num_leaves': 60,
         'metric': 'auc',
         'lambda_l1': 0.1,
         'random_state': 42,
         'verbosity': -1,
         'num_threads' : -1,
           "bagging_freq": 1,
         "bagging_fraction": 0.8 ,

         }
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(folds.split(train_df, train_df.label)):
    
    train = train_df.iloc[train_idx][feature]
    t_label = train_df.iloc[train_idx].label
    valid = train_df.iloc[val_idx][feature]
    v_label = train_df.iloc[val_idx].label
    
    train_dataset = lgb.Dataset(train,label=t_label,categorical_feature=cat_cols)
    valid_dataset = lgb.Dataset(valid,v_label,categorical_feature=cat_cols)

    num_round = 30000
    clf = lgb.train(params,train_dataset,num_round,valid_sets=[train_dataset, valid_dataset],verbose_eval=50,early_stopping_rounds=100)
    predicts[val_idx] = clf.predict(valid,num_iteration=clf.best_iteration)
    pred += clf.predict(test_df[feature], num_iteration=clf.best_iteration)
print(sklearn.metrics.roc_auc_score(train_df.label,predicts))
    

In [ ]:
test_df

In [ ]:
#------------------------------模型预测----------------------------------------#

In [ ]:
pred = pred/5

In [ ]:
0.8092833835

In [ ]:
(pred>1).sum()

In [ ]:
res = pd.DataFrame()
res['id'] = test_df['id'].astype('int32')
res['probability'] = pred
res.to_csv('lgbm.csv',index = False)

In [ ]:
res